In [ ]:
from transformers import BartForConditionalGeneration
from model import GENRE
from trie import Trie, MarisaTrie
import pickle

with open('kilt_titles_trie_dict.pkl', 'rb') as f:
    trie = Trie.load_from_dict(pickle.load(f))

# print(len(list(trie.trie_dict[2].keys())))
# raise ValueError("debug")

# model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
model = GENRE.from_pretrained("hf_entity_disambiguation_aidayago").eval()

sentences = ["Einstein was a [START_ENT] German [END_ENT] physicist."]

result = model.sample(
    sentences,
    prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist())
    )

print(result)


In [1]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer
from trie import MarisaTrie, Trie

model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

# conditional generation
# !!! add bos token to trie !!!
trie = MarisaTrie([[2] + tokenizer.encode('Hello World')])
# print(trie.trie_dict)
output = model.generate(tokenizer.encode('to be or not to be ?', return_tensors='pt'), num_beams=5, max_length=20,
                        prefix_allowed_tokens_fn=lambda batch_id, sent: trie.get(sent.tolist()))
print(tokenizer.batch_decode(output, skip_special_tokens=True))
# original generation
output_n = model.generate(tokenizer.encode('to be or not to be ?', return_tensors='pt'), num_beams=5, max_length=20,)
# print(output_n[0])
print(tokenizer.batch_decode(output_n, skip_special_tokens=True))


/home/patrick/anaconda3/envs/nlp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['Hello World']
['to be or not to be?']
